<a href="https://colab.research.google.com/github/farhanreynaldo/rethinking/blob/master/rethinking_chapter_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load packages used in this notebook
import os
import json
import shutil
import urllib.request
import pandas as pd

In [2]:
# Install package CmdStanPy
!pip install --upgrade cmdstanpy

Requirement already up-to-date: cmdstanpy in /usr/local/lib/python3.6/dist-packages (0.9.62)


In [3]:
# Install pre-built CmdStan binary
# (faster than compiling from source via install_cmdstan() function)
# https://mc-stan.org/users/documentation/case-studies/jupyter_colab_notebooks_2020.html
# https://github.com/stan-dev/example-models/blob/master/knitr/cloud-compute-2020/CmdStanPy_Example_Notebook.ipynb

tgz_file = 'colab-cmdstan-2.23.0.tar.gz'
tgz_url = 'https://github.com/stan-dev/cmdstan/releases/download/v2.23.0/colab-cmdstan-2.23.0.tar.gz'

urllib.request.urlretrieve(tgz_url, tgz_file)
shutil.unpack_archive(tgz_file)

# Specify CmdStan location via environment variable
os.environ['CMDSTAN'] = './cmdstan-2.23.0'

In [4]:
# Check CmdStan path
from cmdstanpy import cmdstan_path
cmdstan_path()

'./cmdstan-2.23.0'

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from cmdstanpy import CmdStanModel

In [6]:
howell1_url = 'https://raw.githubusercontent.com/pymc-devs/resources/master/Rethinking_2/Data/Howell1.csv'
df = pd.read_csv(howell1_url, sep=';', header=0).loc[lambda df: df['age'] >= 18]

In [7]:
data = df.filter(['height']).to_dict(orient='list')
data['n'] = df.shape[0]

In [8]:
%%file m41.stan

data {
  int<lower=1> n;
  vector[n] height;
}
parameters {
  real mu;
  real<lower=0,upper=50> sigma;
}
model {
  height ~ normal(mu, sigma);
  sigma ~ uniform(0, 50);
  mu ~ normal(178, 20);
}

Writing m41.stan


In [17]:
m41 = CmdStanModel(stan_file='m41.stan')
m41_fit = m41.sample(data=data)
m41_fit.summary()

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /content/m41
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:finish chain 4
INFO:cmdstanpy:finish chain 3


,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-895.76000,0.023976,1.021480,-897.90900,-895.4340,-894.8000,1815.14,10550.6,0.999852
mu,154.60500,0.007174,0.420085,153.91200,154.6040,155.2780,3428.70,19929.5,0.999430
sigma,7.78213,0.005300,0.294322,7.31985,7.7702,8.2769,3084.12,17926.5,0.999455


In [18]:
%%file m42.stan

data {
  int<lower=1> n;
  vector[n] height;
}

parameters {
  real mu;
  real<lower=0, upper=50> sigma;
}

model {
  height ~ normal(mu, sigma);
  sigma ~ uniform(0, 50);
  mu ~ normal(178, .1);
}

Overwriting m42.stan


In [20]:
m42 = CmdStanModel(stan_file='m42.stan')
m42_fit = m42.sample(data=data)
m42_fit.summary()

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /content/m42
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:finish chain 4


,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-1301.5900,0.022976,1.002510,-1303.6000,-1301.280,-1300.640,1903.80,11740.0,1.000340
mu,177.8660,0.001624,0.100693,177.6990,177.866,178.031,3843.79,23703.1,1.000530
sigma,24.6037,0.016364,0.933099,23.1425,24.574,26.228,3251.44,20050.3,0.999737
